# Anotación Automática y Entrenamiento de S-BERT usando Contrastive Learning


En este notebook, abordaremos el problema de la falta de datasets completamente anotados mediante un enfoque basado en aprendizaje contrastivo y anotación automática.

### Objetivos:
1. Utilizar un **Golden Dataset** pequeño para entrenar un modelo BERT como cross-encoder.
2. Emplear **aprendizaje contrastivo** para entrenar el cross-encoder, donde la salida indica la similitud entre dos frases.
3. Utilizar el BERT preentrenado para **anotar automáticamente** el resto del dataset.
4. Entrenar un modelo **S-BERT** usando aprendizaje contrastivo para mejorar la calidad de las anotaciones y obtener embeddings más precisos.

Este proceso tiene como objetivo reducir la necesidad de anotaciones manuales y mejorar la calidad del etiquetado del dataset utilizando técnicas avanzadas de NLP.
    

In [ ]:
# Instalamos las librerías necesarias si no están ya instaladas
!pip install transformers sentence-transformers torch datasets -q 

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

## Entrenar BERT como cross-encoder usando un Golden Dataset pequeño

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize pairs of questions
def tokenize_pairs(example):
    # Tokenize the question pairs using the tokenizer with padding and truncation
    tokenized_output = tokenizer(
        example["sentence1"],
        example["sentence2"],
        padding='max_length',
        truncation=True,
        max_length=256  
    )
    # Add the label to the tokenized output and convert it to float type
    tokenized_output["labels"] = [float(label) for label in example["label"]]
    tokenized_output["sentence1"] = example["sentence1"]
    tokenized_output["sentence2"] = example["sentence2"]
    tokenized_output["similarity_score"] = tokenized_output["labels"]
    return tokenized_output



golden_dataset = load_dataset("sentence-transformers/quora-duplicates", "pair-class", split="train[1%:2%]")
golden_dataset = golden_dataset.map(tokenize_pairs, batched=True)

test_dataset = load_dataset("sentence-transformers/quora-duplicates", "pair-class", split="train[2%:3%]")
test_dataset = test_dataset.map(tokenize_pairs, batched=True)

big_train_dataset = load_dataset("sentence-transformers/quora-duplicates", "pair-class", split="train[3%:6%]")
big_train_dataset = big_train_dataset.map(tokenize_pairs, batched=True)

In [ ]:
# Set up the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Configure the training arguments for the Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=100,
    logging_dir='./logs',
    logging_steps=10,
    report_to='none'  # Evitar reportes innecesarios si solo estamos probando  
)

# Initialize the Trainer with the model, arguments, and dataset
trainer_golden = Trainer(
    model=model,
    args=training_args,
    train_dataset=golden_dataset,
    eval_dataset = test_dataset
)

# Train the model using contrastive learning
trainer_golden.train()
print("Training of the cross-encoder BERT completed using the Golden Dataset.")


## Anotación automática del dataset utilizando BERT

In [ ]:
import torch

# Function to annotate the dataset using the trained BERT model
def predict_similarity(example):
    # Tokenize the question pairs
    inputs = tokenizer(
        example["sentence1"],
        example["sentence2"],
        padding='max_length',
        truncation=True,
        max_length=256,
        return_tensors='pt'  # Return PyTorch tensors instead of lists
    )

    # Move inputs to the appropriate device (if you are using a GPU)
    #inputs = {key: value for key, value in inputs.items()}

    # Perform prediction with no gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
        similarity_score = torch.sigmoid(outputs.logits).item()
    inputs["similarity_score"] = similarity_score
    inputs["sentence1"] = example["sentence1"]
    inputs["sentence2"] = example["sentence2"]
    print(inputs["similarity_score"])
    return inputs

# Annotate the rest of the dataset
big_train_dataset = big_train_dataset.map(predict_similarity)
print("Anotación automática del dataset utilizando BERT completada.")


## Entrenar S-BERT usando aprendizaje contrastivo con el dataset anotado

In [5]:
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
big_train_dataset_samples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in big_train_dataset]
golden_train_dataset_samples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in golden_dataset]
test_dataset_samples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score']) for row in test_dataset]

In [ ]:
# Cargar un modelo S-BERT pre-entrenado
sbert_model_golden = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Crear un DataLoader para el entrenamiento con aprendizaje contrastivo
train_dataloader_golden = DataLoader(golden_train_dataset_samples, shuffle=True, batch_size=8)

# Definir la pérdida para el aprendizaje contrastivo
train_loss = losses.CosineSimilarityLoss(model=sbert_model_golden)

# Entrenar el modelo S-BERT utilizando el aprendizaje contrastivo
sbert_model_golden.fit(train_objectives=[(train_dataloader_golden, train_loss)], epochs=1, warmup_steps=100)
print("Entrenamiento del S-BERT con aprendizaje contrastivo completado usando el dataset anotado automáticamente.")
    

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

# Initialize the evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["sentence1"],
    sentences2=test_dataset["sentence2"],
    scores=test_dataset["similarity_score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)
# You can run evaluation like so:
dev_evaluator(sbert_model_golden)

# Ejercicios

### Ejercicio 1

Comparar el entrenamiento de S-BERT con los diferentes datasets (excepto el test). ¿Con cual ofrece mejores resultados?

In [ ]:
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader

# Cargar un modelo S-BERT pre-entrenado
sbert_model_fullTain = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Crear un DataLoader para el entrenamiento con aprendizaje contrastivo
train_dataloader_full = DataLoader(big_train_dataset_samples, shuffle=True, batch_size=8)

# Definir la pérdida para el aprendizaje contrastivo
train_loss = losses.CosineSimilarityLoss(model=sbert_model_fullTain)

# Entrenar el modelo S-BERT utilizando el aprendizaje contrastivo
sbert_model_fullTain.fit(train_objectives=[(train_dataloader_full, train_loss)], epochs=1, warmup_steps=100)
print("Entrenamiento del S-BERT con aprendizaje contrastivo completado usando el dataset anotado automáticamente.")

In [ ]:
dev_evaluator(sbert_model)

### Ejercicio 2

Investigar otros modelos pre-entrenados y evaluarlos con el dataset

In [ ]:
sbert_preTrained = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
# You can run evaluation like so:
dev_evaluator(sbert_preTrained)

### Ejercicio 3

Crear un conjunto de test propio, y evaluar S-BERT en dicho conjunto de datos.  

In [13]:
# Define your custom lists
sentences1 = ["Hace mucho frio","Hace mucho calor"]
sentences2 = ["Hace fresco.","Hace frío."]
similarity_scores = [0.1,0.8]

dev_evaluator_custom = EmbeddingSimilarityEvaluator(
    sentences1=sentences1,
    sentences2=sentences2,
    scores=similarity_scores,
    main_similarity=SimilarityFunction.COSINE,
    name="custom-dataset-evaluator"
)
dev_evaluator_custom(sbert_preTrained)

{'custom-dataset-evaluator_pearson_cosine': 1.0,
 'custom-dataset-evaluator_spearman_cosine': 0.9999999999999999,
 'custom-dataset-evaluator_pearson_manhattan': 1.0,
 'custom-dataset-evaluator_spearman_manhattan': 0.9999999999999999,
 'custom-dataset-evaluator_pearson_euclidean': 1.0,
 'custom-dataset-evaluator_spearman_euclidean': 0.9999999999999999,
 'custom-dataset-evaluator_pearson_dot': 1.0,
 'custom-dataset-evaluator_spearman_dot': 0.9999999999999999,
 'custom-dataset-evaluator_pearson_max': 1.0,
 'custom-dataset-evaluator_spearman_max': 0.9999999999999999}